In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from transformers import BitsAndBytesConfig
import torch

model_id = "huggyllama/llama-7b"
peft_model_id = '../finetune/qlora-tasks-7b-v1.9.2/checkpoint-4500/adapter_model'

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    'huggyllama/llama-7b',
    load_in_4bit=True,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    ),
)
model = PeftModel.from_pretrained(model, peft_model_id)

tokenizer = AutoTokenizer.from_pretrained(model_id)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/boa/.conda/envs/ai/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/boa/.conda/envs/ai/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/boa/.conda/envs/ai/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/boa/.conda/envs/ai/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/boa/.conda/envs/ai/lib/libcudart.so'), PosixPath('/home/boa/.conda/envs/ai/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from datasets import concatenate_datasets, load_from_disk, DatasetDict
combined_tasks = load_from_disk('../datasets/tasks/combined_task_dataset_io_notags/')

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [3]:
combined_tasks['test']['output'][0]

' egyéb\n'

In [4]:
combined_tasks = combined_tasks.filter(lambda data: data['input'].startswith('[korrupció klasszifikáció]'))

Loading cached processed dataset at /home/boa/szakdoga/datasets/tasks/combined_task_dataset_io_notags/train/cache-e6e5446744ce6385.arrow
Loading cached processed dataset at /home/boa/szakdoga/datasets/tasks/combined_task_dataset_io_notags/validation/cache-6cb801e5b02d5abf.arrow
Loading cached processed dataset at /home/boa/szakdoga/datasets/tasks/combined_task_dataset_io_notags/test/cache-6695feed4156056a.arrow


In [5]:
pos_tasks = combined_tasks.filter(lambda data: data['output'] == ' korrupció\n')
neg_tasks = combined_tasks.filter(lambda data: data['output'] != ' korrupció\n')

Loading cached processed dataset at /home/boa/szakdoga/datasets/tasks/combined_task_dataset_io_notags/train/cache-234c7cb357662bf0.arrow
Loading cached processed dataset at /home/boa/szakdoga/datasets/tasks/combined_task_dataset_io_notags/validation/cache-5ccfc1b8aa14da25.arrow
Loading cached processed dataset at /home/boa/szakdoga/datasets/tasks/combined_task_dataset_io_notags/test/cache-996822e76cefe911.arrow
Loading cached processed dataset at /home/boa/szakdoga/datasets/tasks/combined_task_dataset_io_notags/train/cache-72cd976b2dc82047.arrow
Loading cached processed dataset at /home/boa/szakdoga/datasets/tasks/combined_task_dataset_io_notags/validation/cache-dfed212163170834.arrow
Loading cached processed dataset at /home/boa/szakdoga/datasets/tasks/combined_task_dataset_io_notags/test/cache-9ceac2fedd65209e.arrow


In [6]:
pos_tasks

DatasetDict({
    train: Dataset({
        features: ['label', 'theme', 'type', 'input', 'output'],
        num_rows: 1600
    })
    validation: Dataset({
        features: ['label', 'theme', 'type', 'input', 'output'],
        num_rows: 300
    })
    test: Dataset({
        features: ['label', 'theme', 'type', 'input', 'output'],
        num_rows: 150
    })
})

In [7]:
neg_tasks

DatasetDict({
    train: Dataset({
        features: ['label', 'theme', 'type', 'input', 'output'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['label', 'theme', 'type', 'input', 'output'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['label', 'theme', 'type', 'input', 'output'],
        num_rows: 1500
    })
})

In [8]:
pos_tasks['train'] = pos_tasks['train'].select(range(250))
pos_tasks['validation'] = pos_tasks['validation'].select(range(150))
pos_tasks['test'] = pos_tasks['test'].select(range(75))

In [9]:
from datasets import concatenate_datasets, load_dataset, DatasetDict

real_dataset = DatasetDict()

real_dataset['train'] = concatenate_datasets([pos_tasks['train'], neg_tasks['train']])
real_dataset['validation'] = concatenate_datasets([pos_tasks['validation'], neg_tasks['validation']])
real_dataset['test'] = concatenate_datasets([pos_tasks['test'], neg_tasks['test']])

real_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'theme', 'type', 'input', 'output'],
        num_rows: 16250
    })
    validation: Dataset({
        features: ['label', 'theme', 'type', 'input', 'output'],
        num_rows: 3150
    })
    test: Dataset({
        features: ['label', 'theme', 'type', 'input', 'output'],
        num_rows: 1575
    })
})

In [10]:
real_dataset = combined_tasks

In [11]:
from datasets import load_dataset
from datasets import concatenate_datasets

dataset = load_dataset("boapps/kmdb_classification")
test_neg = dataset['test'].filter(lambda row: row['label'] == 0)
test_pos = dataset['test'].filter(lambda row: row['label'] == 1).select(range(len(test_neg)//10))
test_set = concatenate_datasets([test_pos, test_neg]).shuffle(seed=42)

Found cached dataset parquet (/home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_classification-4003d65da9c3e34a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_classification-4003d65da9c3e34a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-5c0dc0418152b777.arrow
Loading cached processed dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_classification-4003d65da9c3e34a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-80ad540b3581070d.arrow
Loading cached shuffled indices for dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_classification-4003d65da9c3e34a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-ee948041db649a27.arrow


In [12]:
article_classification_prompt = '''[korrupció klasszifikáció]
{title}

{description}

cimkék: {keywords}

###

téma: {theme}
'''

In [13]:
def generate_article_classification_task(article):
    article['keywords'] = ', '.join(article['keywords'])
    article['theme'] = ['egyéb', 'korrupció'][article['label']]
    article['description'] = article['description'].replace(article['title'], '').replace(' - ', ' ').strip()
    article['text'] = article['text'].replace(article['description'], '').replace(' - ', ' ').strip()
    article['text'] = article['text'][:article['text'].find('\n')]
    return {'text': article_classification_prompt.format(**article), 'type': 'article_classification'}

    k = 0
#print(generate_article_classification_task(dataset['train'][1])['text'])

In [14]:
test_set = test_set.map(generate_article_classification_task)

Loading cached processed dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_classification-4003d65da9c3e34a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-ac6e9f91cf2d5c36.arrow


In [15]:
test_set = test_set.map(lambda task: {
    'input': task['text'][:task['text'].rfind(':')+1],
    'output': task['text'][task['text'].rfind(':')+1:],
}, remove_columns='text')

Loading cached processed dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_classification-4003d65da9c3e34a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-602f2f47ef90b9d8.arrow


In [16]:
from tqdm import tqdm
device = "cuda:0"

n = 0
k = 0
i = 0

tp = 0
fp = 0
tn = 0
fn = 0

positive = 'korrupció'
negative = 'egyéb'

for task_in, task_out in tqdm(zip(test_set['input'], test_set['output']), total=len(test_set)):
    if not task_in.startswith('[korrupció klasszifikáció]'):
        continue
    with torch.cuda.amp.autocast():
        text = task_in
        inputs = tokenizer(text, return_tensors="pt").to(device)
        if len(inputs['input_ids'][0]) > 2048:
            continue
        outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=4)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        #print(result[result.rfind(':')+2:].strip())
        #print(task_out.strip())
        n += 1
        sresult = result[result.rfind(':')+2:].strip()
        slabel = task_out.strip()
        if sresult not in ['korrupció', 'egyéb']:
            i += 1
        if sresult == positive and slabel == positive:
            tp += 1
        elif sresult == negative and slabel == negative:
            tn += 1
        elif sresult == positive and slabel == negative:
            fp += 1
        elif sresult == negative and slabel == positive:
            fn += 1
        if task_out.strip() == (result[result.rfind(':')+2:].strip()):
            k += 1
    if n % 10 == 0 and n > 0 and tp+fp > 0 and tp+fn > 0:
        print(k/n, k, n, i)
        print('precision', tp/(tp+fp))
        print('recall', tp/(tp+fn))
        print('accuracy', (tp+tn)/(tp+tn+fp+fn))
        print(tp, tn, fp, fn)


print(k/n, k, n, i)

print(tp, tn, fp, fn)

  0%|▌                                                                                                                            | 10/2084 [00:07<23:15,  1.49it/s]

0.9 9 10 0
precision 1.0
recall 0.5
accuracy 0.9
1 8 0 1


  1%|█▏                                                                                                                           | 20/2084 [00:13<20:33,  1.67it/s]

0.95 19 20 0
precision 1.0
recall 0.8
accuracy 0.95
4 15 0 1


  1%|█▊                                                                                                                           | 30/2084 [00:19<20:41,  1.65it/s]

...

0.9705314009661836 2009 2070 0
precision 0.8587570621468926
recall 0.8085106382978723
accuracy 0.9705314009661836
152 1857 25 36


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 2080/2084 [21:23<00:02,  1.64it/s]

0.9706730769230769 2019 2080 0
precision 0.8595505617977528
recall 0.8095238095238095
accuracy 0.9706730769230769
153 1866 25 36


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2084/2084 [21:26<00:00,  1.62it/s]

0.9707293666026872 2023 2084 0
153 1870 25 36


In [17]:
print('precision', tp/(tp+fp))
print('recall', tp/(tp+fn))
print('accuracy', (tp+tn)/(tp+tn+fp+fn))

precision 0.8595505617977528
recall 0.8095238095238095
accuracy 0.9707293666026872
